In [47]:
import geopandas as gpd
from geojson import GeoJSON
from numpy import pi, cos, sin
from shapely.geometry import Polygon
import json

In [48]:


def single_hex(x, y, r):
    verts = []
    for i in range(6):
        angle = i * 2 * pi / 6
        xc = x + cos(angle) * r
        yc = y + sin(angle) * r
        verts.append((xc, yc))
    return verts


def make_grid(center, radius_count=20, hex_radius=0.005):
    results = []
    hex_list = []
    hex_list.append(single_hex(center[0],center[1], hex_radius))
    for n in range(1,radius_count,1): # get the corner hexes of a band this distance from center
        c2c = n*2*hex_radius*sin(pi/3) # distance of center of these verts from overall center
        verts = []
        for m in range(6):
            angle = ((m/6)-(1/12)) * (2.0*pi)
            v_x = center[0] + cos(angle)*c2c # coordinates of one of the six corner hexes
            v_y = center[1] + sin(angle)*c2c
            verts.append((v_x, v_y))
        for i,this_hex in enumerate(verts):# fill in the hexes between the corners
            next_hex = verts[(i+1)%6] # this allows wraparound when finding the next hex counterclockwise
            centers = []
            side_length = n
            gap_x = (next_hex[0]-this_hex[0])/side_length
            gap_y = (next_hex[1]-this_hex[1])/side_length
            for j in range(side_length):
                x_coord = this_hex[0]+gap_x*j
                y_coord = this_hex[1]+gap_y*j
                hex_list.append(single_hex(x_coord,y_coord,hex_radius))
    for hex_cords in hex_list:
        results.append(Polygon(hex_cords))
    return results





In [53]:

class city:
    def __init__(self, name, latitude, longitude, radius = 20, hex_size = 0.005):
        # a word to the wise, latitude, the y-coordinate, conventionally comes first.  West is negative longitude.
        self.name = str(name)
        self.latitude = float(latitude)
        self.longitude = float(longitude)
        cities.append(self)
        self.radius = radius
        self.hex_size = hex_size
    def hexify(self):
        self.shapes = gpd.GeoDataFrame(make_grid((self.latitude,self.longitude, self.radius)), columns=['geometry'])
        self.shapes['hex_number'] = self.shapes.index
    def self_to_json(self):        
        with open(self.name+'_hex.json', 'w') as outfile:
            json.dump(json.loads(self.shapes.to_json()), outfile)
        
    def clear_frame(self):
        self.shapes = None
        


In [54]:
cities = []
city('Washington', latitude=-77.0369, longitude=38.9072, radius=40, hex_size = 0.0025)
city('San_Francisco', latitude = -122.4194, longitude = 37.7749, radius=40, hex_size = 0.0025)
city('New_York', latitude=-74.0060, longitude = 40.7128, radius=40, hex_size = 0.0025)

cities

In [55]:
for burb in cities:
    burb.hexify()
    burb.self_to_json()
    burb.clear_frame()